In [1]:
import MeCab
import pandas as pd
import re
import itertools
import math
import numpy as np
from src.vectrizer import ContentVectorizer
from src.clustering import ClustarGenerator
from src.network_generater import NetworkGenerater

cv = ContentVectorizer()
cg = ClustarGenerator()
ng = NetworkGenerater()

# ユーザー辞書を設定
m_t = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -u data/fragrance_user_dic.dic')

# メタ付与する文章を読み込み
article_df = pd.read_table('data/perfume_sentence.tsv')

colname = ['surface', 'pos1', 'pos2', 'empty1', 'empty2', 'empty3', 'empty4', 'nomarization', 'category1', 'empty5', 'empty6', 'empty7', 'empty8', 'empty9', 'pos3', 'empty10', 'sign']

ModuleNotFoundError: No module named 'japanize_matplotlib'

In [ ]:
# 学習済みベクトルを差し替える

#model_path = "model/awg_word2vec.model"
#cv.load_model(model_path)
#cv.ginza.vocab.vectors.shape

# 文章から抽出したワードのベクトルを作成する

以下のようなテーブルを作成し、`特徴語リスト`の単語をそれぞれベクトル化する。

|  記事文章  |  特徴語リスト  |
| ---- | ---- |
|  記事文章1  |  特徴語,特徴語,特徴語…  |
|  記事文章2  |  特徴語,特徴語,特徴語…  |
|  ...  |  ...  |

ベクトル化できなかった単語は`empty_w`リストに入れておく。使用する学習モデルの性能確認用。

In [ ]:
# 辞書から特定のカテゴリーに属するワードを抽出する
feature = [] # 記事ごとに特徴語リストを格納
word_class = [] # 記事の文章から抽出した一意のワードを格納
wlstr_ = [] # 一時変数
empty_w = [] # ベクトル化できない単語を格納（ゼロベクトルのリスト）

for i, sentence in enumerate(article_df['description']):
    parsed_s = m_t.parse(sentence).replace('\t',',').split('\n')
    parsed_results = pd.Series(parsed_s).str.split(',').tolist()
    df_raw = pd.DataFrame(parsed_results, columns = colname)
    # 'category1'に「香り」か「印象」属性が登録されている単語をDataFarameに登録
    f_df = df_raw[(df_raw['category1'] == '香り') | (df_raw['category1'] =='印象')]

    # 単語と香りor印象属性を抽出する
    f_m = []
    attr_dic = {}
    for s,c in zip(f_df['surface'],f_df['category1']):
        doc,vector = cv.vectorize(s)
        attr_dic = {'word':doc,'category':c}
        if vector.all():
            if str(doc) not in wlstr_:
                word_class.append(doc)
                wlstr_.append(str(doc))
            f_m.append(attr_dic)
        else:
            empty_w.append(doc)
    feature.append(f_m)
    
# 出力
print('ベクトル化できなかった単語:',empty_w)

# 特徴語の類似度テーブルを作成する

|  特徴語A  |  特徴語B  |  類似度(スコア)  |
| ---- | ---- | ---- |
|  特徴語  |  特徴語  |  0.356119  |
|  特徴語  |  特徴語  |  0.464299  |
|  ...  |  ...  |  ...  |

特徴語同士がどれくらい似ているか比較します。1に近いほど2つの特徴語が似ていることになります。  
ただし、学習モデルに登録されていないワードは抜き出せません。  
例：「ダマスクローズ」というワードは学習モデルに存在しないため、「ダマスク」と「ローズ」それぞれのベクトルの平均値が「ダマスクローズ」のベクトルとして出力されます。  
　　「ダマスク」というワードが存在しない場合は「ローズ」のベクトルが出力されます。

In [ ]:
# 特徴語の組み合わせを全パターン作成する
meta_parts_pairs = []
for pair in itertools.combinations(word_class, 2):
    meta_parts_pairs.append(pair)
    
# 特徴語の類似度（ベクトル同士の類似度）を計算する
score = []
for doc1,doc2 in meta_parts_pairs:
    score.append(doc1.similarity(doc2))
    
# 上記の結果をテーブルに格納
target_df = pd.DataFrame(meta_parts_pairs)
target_df['2']=score

# 出力
target_df

# クラスタリング
k-meansで特徴語をクラスタリングします。  
参考：[k-means法を理解する](https://qiita.com/g-k/items/0d5d22a12a4507ecbf11)

- `n_cluster`: クラスタ数を設定できます。  
- `cluster_result`: {クラスタID:特徴語リスト}な辞書です。

In [ ]:
vectorlist = np.array([w.vector for w in word_class])

# クラスタ数の設定
n_cluster = 10

cluster_result = cg.generate(title_list=word_class, 
            vector_list=vectorlist, 
            vector_size=100, 
            n_cluster = n_cluster)

print(cluster_result)

# クラスタごとのネットワークを作成
- `cluster_result_dict`: {クラスタID:特徴語同士の距離テーブル}な辞書を作成します。グラフ構造を作成するために使います。
- `cluster_df['2']>0.5`でスコアのしきい値を設定します。デフォルトでは0.5以下のスコアは表示されません。

In [ ]:
cluster_word = cluster_result[0]

cluster_result_dict = {}
for class_id , cluster_word in cluster_result.items():
    meta_parts_pairs = []
    for pair in itertools.combinations(cluster_word, 2):
        meta_parts_pairs.append(pair)

    score = []
    for doc1,doc2 in meta_parts_pairs:
        score.append(round(doc1.similarity(doc2),3))

    cluster_df = pd.DataFrame(meta_parts_pairs)
    cluster_df['2']=score

    cluster_result_dict[class_id]=cluster_df[cluster_df['2']>0.5].sort_values('2',ascending=False)

# ネットワーク可視化
クラスタごとに特徴語のネットワークを作成します。

In [ ]:
for clusterid, cluster in cluster_result_dict.items():
    print('Cluster ID: 'clusterid)
    ng.network(cluster.values.tolist(), 200, 0.005)